In [ ]:
from uproot_module import Data
from vbfdm import VBFDM
import uproot
import awkward
import pandas as pd
import numpy as np

In [ ]:
SIGNAL_PATH = "/home/santiago/VBF_DMSimp_spin0_EWKExcluded/Events/run_18/DMSimpSpin0_MY5000_MX1000_07042020.root"
TREE_NAME = "Delphes"
N_JETS = 6 #Número de jets

pt = ("Jet.PT[%d]"%(i) for i in range(N_JETS))
phi = ("Jet.Phi[%d]"%(i) for i in range(N_JETS))
eta = ("Jet.Eta[%d]"%(i) for i in range(N_JETS))
d = Data(*(*pt,*phi,*eta,"MissingET.MET","MissingET.Phi"))
d.setSignal(SIGNAL_PATH, TREE_NAME)
d.dataframe.head()

In [ ]:
def DeltaPhi(phi1,phi2):
    """
    Returns the difference of phi1 and phi2
    """
    phi = phi1-phi2
    for i in range(phi.shape[0]):
        if phi[i] >= np.pi:
            phi[i] -= 2*np.pi
        elif phi[i] < -1*np.pi:
            phi[i] += 2*np.pi
    return phi

# Add new columns to data frame
d.dataframe["HT"] = d.dataframe[["Jet.PT[{}]".format(i) for i in range(N_JETS)]].sum(axis=1)

# Delta Phi between jets
for i in range(N_JETS):
    for j in range(i+1,N_JETS):
        d.dataframe[f"DPhi_J{i}_J{j}"] = DeltaPhi(d.dataframe[f"Jet.Phi[{i}]"],d.dataframe[f"Jet.Phi[{j}]"])


d.dataframe.head()

In [ ]:
# Cuts
def cut1(df):
    # H > 200
    mask = df["HT"] > 200
    df = df.loc[mask,:]
    return df

def cut2(df):
    # at least two jets per event 
    mask = df["Jet.PT[0]"].notnull() & df["Jet.PT[1]"].notnull()
    df = df.loc[mask,:]
    return df

def cut3(df):
    # pt > 30 GeV and |eta| < 5 for leading and subleading jets
    mask = ((df["Jet.PT[0]"] > 30) & 
            (df["Jet.PT[1]"] > 30) & 
            (abs(df["Jet.Eta[0]"]) < 5) &
            (abs(df["Jet.Eta[1]"]) < 5))
    df = df.loc[mask,:]
    return df

def cut4(df):
    # leading jets in opposite hemispheres
    mask = df["Jet.Eta[0]"] * df["Jet.Eta[1]"] < 0
    df = df.loc[mask,:]
    return df

def cut5(df):
    # |DeltaPhi| >= 2.3 for leading jets
    mask = abs(df["DPhi_J0_J1"]) >= 2.3
    df = df.loc[mask,:]
    return df

def cut6(df):
#     # max invariant mass >= 1000
#     mask = df.max_inv_mass >= 1000
#     df = df.loc[mask,:]
    return df

def cut7(df):
    index = []
    for i in range(N_JETS):
        for j in range(i+1,N_JETS):
            index.append((i,j))
            
    mask = np.abs(df[[f"DPhi_J{i}_J{j}" for i,j in index]]).min(axis=1) > 0.5
    df = df.loc[mask,:]
    return df

In [ ]:
type(d.dataframe[["Jet.PT[{}]".format(i) for i in range(N_JETS)]].sum(axis=1))

In [ ]:
d.addCut("cut1",cut1)
d.addCut("cut2",cut2)
d.addCut("cut3",cut3)
d.addCut("cut4",cut4)
d.addCut("cut5",cut5)
d.addCut("cut6",cut6)
d.addCut("cut7",cut7)

d.cutFlow()

In [ ]:
dic = d.cutFlow()
pd.Series(dic)

In [ ]:
SIGNAL_PATH = "/home/santiago/VBF_DMSimp_spin0_EWKExcluded/Events/run_18/DMSimpSpin0_MY5000_MX1000_07042020.root"

vbf = VBFDM(SIGNAL_PATH, SIGNAL_PATH)

In [ ]:
vbf.cut_flow()